In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class DataLoaderConfig:
    root_dir: Path
    train_dir: Path
    test_dir: Path
    image_transforms_path: Path
    weights: str
    params_batch_size: int

In [5]:
from cnnClassifier.utils.common import read_yaml, create_dir
from cnnClassifier.constants import *

In [6]:
class ConfigurationManager:
    def __init__(self) -> None:
        config_path = CONFIG_PATH
        params_path = PARAMS_PATH
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_dir([self.config['artifacts_root']])
    
    def get_dataloader_config(self) -> DataLoaderConfig:
        config = self.config['dataloader']
        create_dir([config['root_dir']])
        
        dataloader_config = DataLoaderConfig(
            root_dir=config['root_dir'],
            train_dir=config['train_dir'],
            test_dir=config['test_dir'],
            image_transforms_path=config['image_transforms_path'],
            weights=self.params['WEIGHTS'],
            params_batch_size=self.params['BATCH_SIZE']
        )
        return dataloader_config

In [7]:
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader as TorchDataLoader
from cnnClassifier.utils.common import save_model

In [8]:
class Dataloader:
    def __init__(self, config: DataLoaderConfig):
        self.config = config
        
        
    def create_dataloader(self):
        weights = torchvision.models.get_weight(f"{self.config.weights}.DEFAULT")
        transforms = weights.transforms()
        train_data = ImageFolder(
            root=self.config.train_dir,
            transform=transforms
        )
        test_data = ImageFolder(
            root=self.config.test_dir,
            transform=transforms
        )
        
        classes_name = train_data.classes
        
        train_dataloader = TorchDataLoader(
            dataset=train_data,
            batch_size=self.config.params_batch_size,
            shuffle=True
        )
        test_dataloader = TorchDataLoader(
            dataset=test_data,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )
        # save transforms image
        save_model(transforms, self.config.image_transforms_path)
        self.save_classes(classes_name)
        return train_dataloader, test_dataloader, classes_name
    
    def save_classes(self, classes_name):
        with open('classes.txt', 'w') as file:
            for breed in classes_name:
                file.write(f"{breed}\n")

In [9]:
try:
    config = ConfigurationManager()
    dataloader_config = config.get_dataloader_config()
    dataloader = Dataloader(config=dataloader_config)
    train_dataloader, test_dataloader, classes = dataloader.create_dataloader()
    print(train_dataloader, test_dataloader, classes)
except Exception as e:
    raise

[2024-01-12 22:01:07,771: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2024-01-12 22:01:07,773: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-01-12 22:01:07,774: INFO: common: Created directory at: artifacts]
[2024-01-12 22:01:07,775: INFO: common: Created directory at: artifacts/dataloader]
['Afghan', 'African Wild Dog', 'Airedale', 'American Hairless', 'American Spaniel', 'Basenji', 'Basset', 'Beagle', 'Bearded Collie', 'Bermaise', 'Bichon Frise', 'Blenheim', 'Bloodhound', 'Bluetick', 'Border Collie', 'Borzoi', 'Boston Terrier', 'Boxer', 'Bull Mastiff', 'Bull Terrier', 'Bulldog', 'Cairn', 'Chihuahua', 'Chinese Crested', 'Chow', 'Clumber', 'Cockapoo', 'Cocker', 'Collie', 'Corgi', 'Coyote', 'Dalmation', 'Dhole', 'Dingo', 'Doberman', 'Elk Hound', 'French Bulldog', 'German Sheperd', 'Golden Retriever', 'Great Dane', 'Great Perenees', 'Greyhound', 'Groenendael', 'Irish Spaniel', 'Irish Wolfhound', 'Japanese Spaniel', 'Komondor', 'Labradoodle', 'Labra